In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os

In [2]:
data_dir = "/home/armoraux/Python/pysteps_eval/github/data"

In [3]:
pysteps_fp = os.path.join(data_dir, "pysteps/blended_nowcast_202509090000.nc")
pysteps_ds = xr.open_dataset(pysteps_fp, engine='h5netcdf')
pysteps_ds

<xarray.Dataset> Size: 3GB
Dimensions:           (x: 700, y: 700, ens_number: 24, time: 72)
Coordinates:
  * x                 (x) float32 3kB 3.005e+05 3.015e+05 ... 9.995e+05
  * y                 (y) float32 3kB 9.995e+05 9.985e+05 ... 3.005e+05
  * ens_number        (ens_number) int64 192B 1 2 3 4 5 6 ... 19 20 21 22 23 24
  * time              (time) datetime64[ns] 576B 2025-09-09T00:05:00 ... 2025...
Data variables:
    lon               (y, x) float64 4MB ...
    lat               (y, x) float64 4MB ...
    lcc               int64 8B ...
    precip_intensity  (ens_number, time, y, x) float32 3GB ...
Attributes:
    Conventions:  CF-1.7
    title:        pysteps-generated nowcast
    institution:  the pySTEPS community (https://pysteps.github.io)
    source:       pysteps
    history:      
    references:   
    comment:      
    projection:   +proj=lcc +lat_1=49.83333333333334 +lat_2=51.16666666666666...

In [4]:
import xarray as xr
import pyproj
import numpy as np
import pandas as pd
import time
import pysteps
from pysteps.io.importers import import_odim_hdf5 as importer
from pysteps.io.archive import find_by_date
from pysteps.io.readers import read_timeseries
import datetime as dt
import h5py

def read_radar(datetimes):
    """Read RADQPE precipitation rate data for the specified dates, and return them as xr.Dataset 
    with the associated metadata.
    
    Args:
        datetimes (xr.DataArray): Array of datetimes for the RADQPE data to be read.
    
    Returns:
        ds (xr.Dataset): Dataset of the RADQPE precipitation rate data, with the associated metadata.
    """
    t0 = time.time()
    print("Reading RADAR data...", end="", flush=True)
    # get the radar paths and filename formats
    rmi = pysteps.rcparams.data_sources.rmi
    # root_path = rmi.root_path
    root_path = "/mnt/RADQPE2/archive/"
    path_fmt = rmi.path_fmt
    fn_pattern = rmi.fn_pattern
    fn_ext = rmi.fn_ext
    importer_kwargs = rmi.importer_kwargs
    startdate = datetimes.isel(time=0).data
    enddate = datetimes.isel(time=len(datetimes) - 1).data
    # find the number of previous timesteps
    timestep = (datetimes.isel(time=1).data - startdate) / np.timedelta64(1, "m")
    nprev = int((enddate - startdate) / np.timedelta64(1, "m") / timestep)
    # build the filenames
    print(enddate)
    fns = find_by_date(
        dt.datetime.utcfromtimestamp(int(enddate) / 1e9),
        root_path,
        path_fmt,
        fn_pattern,
        fn_ext,
        timestep,
        num_prev_files=nprev
    )
    
    return fns

Pysteps configuration file found at: /home/armoraux/miniconda3/envs/xrValidation/lib/python3.11/site-packages/pysteps/pystepsrc



In [6]:
radar_path_dict = {
    "root_path": os.path.join(data_dir, "radar"),
    "path_fmt": "%Y/%m/%d/best/comp/rate/qpe_edk/hdf",
    "fn_pattern": "%Y%m%d%H%M00.rad.best.comp.rate.qpe_edk",
    "fn_ext": "hdf",
    "importer_kwargs": {'accutime': 5.0, 'ignore_gain_offset': True}
    }

radar_fps = read_radar(pysteps_ds.time, radar_path_dict)[0]
radar_fps

TypeError: read_radar() takes 1 positional argument but 2 were given

In [ ]:
import shutil

In [ ]:
for radar_fp in radar_fps:
    radar_fn = radar_fp.split('/')[-1]
    radar_fp2 = os.path.join(data_dir, "radar", radar_fn)
    shutil.copy(radar_fp, radar_fp2)